# Getting started with BigQuery GIS #

Remake of the example https://cloud.google.com/bigquery/docs/gis-getting-started with Lets-Plot library

### Note ###

Before start you should create your own service account JSON key and put it into data/ folder named `bigquery_credentials.json`.

Some instructions:
- [How to create project](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
- [How to create service account](https://cloud.google.com/iam/docs/creating-managing-service-accounts)
- [How to get service account key](https://cloud.google.com/docs/authentication/getting-started)

In [1]:
import geopandas as gpd
from shapely import wkt
from google.oauth2 import service_account
from google.cloud import bigquery

from lets_plot import *
LetsPlot.setup_html()
from lets_plot.settings_utils import maptiles_lets_plot

In [2]:
QUERY = '''
    SELECT ST_GeogPoint(longitude, latitude) as WKT, num_bikes_available
    FROM `bigquery-public-data.new_york.citibike_stations`
    WHERE num_bikes_available > 30
'''

In [3]:
credentials = service_account.Credentials.from_service_account_file(
    '../data/bigquery_credentials.json',
    scopes=['https://www.googleapis.com/auth/cloud-platform'],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

FileNotFoundError: [Errno 2] No such file or directory: '../data/bigquery_credentials.json'

In [ ]:
df = client.query(QUERY).to_dataframe()
df['WKT'] = df['WKT'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='WKT')

In [ ]:
ggplot() + \
    geom_livemap(zoom=11) + \
    geom_point(aes(size='num_bikes_available'), data=gdf, color='#0000ff', alpha=.5) + \
    scale_size(range=[4, 8], name='available bikes') + \
    ggtitle('Number of Bikes Available at the Location of NYC') + \
    ggsize(600, 450)